In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import os
from joblib import parallel_backend
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from FaceBinaryDataset import FaceBinaryDataset
from build_pipeline import build_pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
dataset = FaceBinaryDataset("dataset/output_dataset", dimension=160)

images = []
labels = []

for i in range(len(dataset)):
    img, label, userid = dataset[i]

    if img is None:
        continue

    images.append(img)
    labels.append(label)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

No face detected in: dataset/output_dataset\authorized\user_00001\img_005.jpg
No face detected in: dataset/output_dataset\authorized\user_00001\img_007.jpg
No face detected in: dataset/output_dataset\authorized\user_00001\img_011.jpg
No face detected in: dataset/output_dataset\authorized\user_00001\img_012.jpg
No face detected in: dataset/output_dataset\authorized\user_00001\img_013.jpg
No face detected in: dataset/output_dataset\authorized\user_00001\img_015.jpg
No face detected in: dataset/output_dataset\authorized\user_00002\img_009.jpg
No face detected in: dataset/output_dataset\authorized\user_00003\img_001.jpg
No face detected in: dataset/output_dataset\authorized\user_00003\img_006.jpg
No face detected in: dataset/output_dataset\authorized\user_00003\img_007.jpg
No face detected in: dataset/output_dataset\authorized\user_00003\img_008.jpg
No face detected in: dataset/output_dataset\authorized\user_00003\img_009.jpg
No face detected in: dataset/output_dataset\authorized\user_0000

# Hyperparameter Tuning

In [10]:
cv = RepeatedStratifiedKFold(
    n_splits=2,      # 2‐fold
    n_repeats=5,     # repeated 5 times
    random_state=42  # for reproducibility
)

param_grid = {
    'features__extractor__num_points': [8, 16, 24],
    'features__extractor__radius'    : [1, 2, 8],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 100]
}

out_dir = "C:/Users/macie/PycharmProjects/face-recognition/models"
os.makedirs(out_dir, exist_ok=True)

# LBP

In [8]:
pipeline_LBP = build_pipeline("lbp")
pipeline_LBP.fit(X_train, y_train)

C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\applications\mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(
C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\feature\texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Pipeline(steps=[('features', FeatureExtractor(extractor=LBPExtractor())),
                ('standardScaler', StandardScaler()), ('classifier', SVC())])

In [11]:
grid_search_SVC = GridSearchCV(
        pipeline_LBP,
        param_grid=param_grid,
        cv=5,
        verbose=2,
        n_jobs=-1,
    )

grid_search_SVC.fit(X_train, y_train)
LBP_best = grid_search_SVC.best_estimator_
joblib.dump(LBP_best, os.path.join(out_dir, "SVC_lbp.pkl"))

Fitting 5 folds for each of 54 candidates, totalling 270 fits


C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\feature\texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


['C:/Users/macie/PycharmProjects/face-recognition/models\\SVC_lbp.pkl']

In [12]:
print(grid_search_SVC.best_params_)
print(grid_search_SVC.best_score_)

{'classifier__C': 1, 'classifier__kernel': 'rbf', 'features__extractor__num_points': 24, 'features__extractor__radius': 2}
0.5656744790222806


# HOG

In [4]:
param_grid = {
    'features__extractor__orientations': [9, 12],
    'features__extractor__pixels_per_cell': [(8, 8), (16, 16)],
    'features__extractor__cells_per_block': [(2, 2), (3, 3)],
    
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 100]
}
pipeline_HOG = build_pipeline("hog")
pipeline_HOG.fit(X_train, y_train)

C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\applications\mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Pipeline(steps=[('features', FeatureExtractor(extractor=HOGExtractor())),
                ('standardScaler', StandardScaler()), ('classifier', SVC())])

In [5]:
grid_search_SVC = GridSearchCV(
        pipeline_HOG,
        param_grid=param_grid,
        cv=5,
        verbose=2,
        n_jobs=-1,
    )
grid_search_SVC.fit(X_train, y_train)
HOG_best = grid_search_SVC.best_estimator_
joblib.dump(HOG_best, os.path.join(out_dir, "SVC_hog.pkl"))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


['C:/Users/macie/PycharmProjects/face-recognition/models\\SVC_hog.pkl']

In [6]:
print(grid_search_SVC.best_params_)
print(grid_search_SVC.best_score_)

{'classifier__C': 100, 'classifier__kernel': 'rbf', 'features__extractor__cells_per_block': (3, 3), 'features__extractor__orientations': 9, 'features__extractor__pixels_per_cell': (16, 16)}
0.6269981703405068


# FACENET

In [4]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 100]
}
pipeline_FN = build_pipeline("facenet")
pipeline_FN.fit(X_train, y_train)

C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\applications\mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Pipeline(steps=[('features',
                 FeatureExtractor(extractor=<extraction.FaceNetExtractor object at 0x00000178608F6EA0>)),
                ('standardScaler', StandardScaler()), ('classifier', SVC())])

In [5]:
grid_search_SVC = GridSearchCV(
        pipeline_FN,
        param_grid=param_grid,
        cv=5,
        verbose=2,
        n_jobs=-1,
    )
# with parallel_backend('threading', n_jobs=-1):
grid_search_SVC.fit(X_train, y_train)
FN_best = grid_search_SVC.best_estimator_
joblib.dump(FN_best, os.path.join(out_dir, "SVC_facenet.pkl"))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


['C:/Users/macie/PycharmProjects/face-recognition/models\\SVC_facenet.pkl']

In [6]:
print(grid_search_SVC.best_params_)
print(grid_search_SVC.best_score_)

{'classifier__C': 100, 'classifier__kernel': 'rbf'}
0.8972974899606345


# CNN

In [7]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 100]
}
pipeline_CNN = build_pipeline("cnn")
pipeline_CNN.fit(X_train, y_train)

C:\Users\macie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\applications\mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━

Pipeline(steps=[('features',
                 FeatureExtractor(extractor=<extraction.CNNExtractor object at 0x0000017869B277D0>)),
                ('standardScaler', StandardScaler()), ('classifier', SVC())])

In [8]:
grid_search_SVC = GridSearchCV(
        pipeline_CNN,
        param_grid=param_grid,
        cv=5,
        verbose=2,
        n_jobs=-1,
    )
# with parallel_backend('threading', n_jobs=-1):
grid_search_SVC.fit(X_train, y_train)
CNN_best = grid_search_SVC.best_estimator_
joblib.dump(CNN_best, os.path.join(out_dir, "SVC_cnn.pkl"))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 981ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

['C:/Users/macie/PycharmProjects/face-recognition/models\\SVC_cnn.pkl']

In [9]:
print(grid_search_SVC.best_params_)
print(grid_search_SVC.best_score_)

{'classifier__C': 100, 'classifier__kernel': 'rbf'}
0.5533579400093462
